In [1]:
import pandas as pd
import numpy as np
from sklearn.covariance import EllipticEnvelope
from sklearn.svm import OneClassSVM
from joblib import dump, load
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
pred_cols = ["Mean_BMI","Median_BMI","Unmet_Need_Rate","Under5_Mortality_Rate","Skilled_Birth_Attendant_Rate","Stunted_Rate"]

In [3]:
train = pd.read_parquet('../data/train.parquet.gzip')
train = train[~train.index.duplicated(keep='first')]
low_imp_features = load('../data/low_imp_features.joblib')
train.drop(columns=low_imp_features,inplace=True)

In [4]:
X = train.drop(pred_cols, axis=1)
y = train[pred_cols]
X

,Es_min_max@CAS/IGSNRR/PML/V2&timestamped,basic_demographic_characteristics_min_max@CIESIN/GPWv411/GPW_Basic_Demographic_Characteristics&timestamped,national_identifier_grid@CIESIN/GPWv411/GPW_National_Identifier_Grid,national_identifier_grid_max@CIESIN/GPWv411/GPW_National_Identifier_Grid,national_identifier_grid_min@CIESIN/GPWv411/GPW_National_Identifier_Grid,urban-coverfraction_mean@COPERNICUS/Landcover/100m/Proba-V-C3/Global&timestamped,SO2_column_number_density_15km_max_max@COPERNICUS/S5P/NRTI/L3_SO2&timestamped,solar_azimuth_angle_max_max@COPERNICUS/S5P/NRTI/L3_SO2&timestamped,ozone_tropospheric_mixing_ratio_median@COPERNICUS/S5P/OFFL/L3_O3_TCL&timestamped,ozone_tropospheric_vertical_column_median@COPERNICUS/S5P/OFFL/L3_O3_TCL&timestamped,...,onehotencoder__DHSCC_DR,onehotencoder__DHSCC_ET,onehotencoder__DHSCC_HT,onehotencoder__DHSCC_MZ,onehotencoder__DHSCC_TZ,onehotencoder__URBAN_RURA_R,onehotencoder__URBAN_RURA_U,remainder__DHSYEAR,remainder__LATNUM,remainder__LONGNUM
DHSID,,,,,,,,,,,,,,,,,,,,,
AL200800000001,0.079005,0.533688,8.0,8.0,8.0,15.942918,0.001929,179.557310,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2008.0,40.822650,19.838322
AL200800000002,0.090663,0.165650,8.0,8.0,8.0,1.059678,0.001774,179.592150,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2008.0,40.696846,20.007555
AL200800000004,0.085710,0.814697,8.0,8.0,8.0,10.031118,0.001929,179.768220,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2008.0,40.798930,19.863338
AL200800000010,0.105490,8.942595,8.0,8.0,8.0,22.716291,0.001986,179.742550,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2008.0,40.698520,19.950300
AL200800000011,0.106758,8.940361,8.0,8.0,8.0,41.684960,0.001609,179.742550,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2008.0,40.717968,19.935875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZW201500000395,0.007476,0.000000,716.0,716.0,716.0,0.000000,0.000404,-18.019940,51.753593,0.011510,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2015.0,-17.166506,29.718370
ZW201500000396,0.002424,0.000000,716.0,716.0,716.0,80.522360,0.000402,-17.357134,53.740124,0.011052,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2015.0,-17.915289,31.156115
ZW201500000397,0.038481,0.000000,716.0,716.0,716.0,0.298818,0.000336,-16.403381,53.757790,0.010933,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2015.0,-18.379501,31.872288


In [5]:
# ee = EllipticEnvelope()
# yhat = ee.fit_predict(X)

In [6]:
ee = OneClassSVM()
yhat = ee.fit_predict(X)

In [7]:
mask = yhat != -1
mask.shape

(36680,)

In [8]:
X_new, y_new = X[mask], y[mask]
X_new

,Es_min_max@CAS/IGSNRR/PML/V2&timestamped,basic_demographic_characteristics_min_max@CIESIN/GPWv411/GPW_Basic_Demographic_Characteristics&timestamped,national_identifier_grid@CIESIN/GPWv411/GPW_National_Identifier_Grid,national_identifier_grid_max@CIESIN/GPWv411/GPW_National_Identifier_Grid,national_identifier_grid_min@CIESIN/GPWv411/GPW_National_Identifier_Grid,urban-coverfraction_mean@COPERNICUS/Landcover/100m/Proba-V-C3/Global&timestamped,SO2_column_number_density_15km_max_max@COPERNICUS/S5P/NRTI/L3_SO2&timestamped,solar_azimuth_angle_max_max@COPERNICUS/S5P/NRTI/L3_SO2&timestamped,ozone_tropospheric_mixing_ratio_median@COPERNICUS/S5P/OFFL/L3_O3_TCL&timestamped,ozone_tropospheric_vertical_column_median@COPERNICUS/S5P/OFFL/L3_O3_TCL&timestamped,...,onehotencoder__DHSCC_DR,onehotencoder__DHSCC_ET,onehotencoder__DHSCC_HT,onehotencoder__DHSCC_MZ,onehotencoder__DHSCC_TZ,onehotencoder__URBAN_RURA_R,onehotencoder__URBAN_RURA_U,remainder__DHSYEAR,remainder__LATNUM,remainder__LONGNUM
DHSID,,,,,,,,,,,,,,,,,,,,,
AL200800000001,0.079005,0.533688,8.0,8.0,8.0,15.942918,0.001929,179.557310,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2008.0,40.822650,19.838322
AL200800000002,0.090663,0.165650,8.0,8.0,8.0,1.059678,0.001774,179.592150,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2008.0,40.696846,20.007555
AL200800000004,0.085710,0.814697,8.0,8.0,8.0,10.031118,0.001929,179.768220,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2008.0,40.798930,19.863338
AL200800000010,0.105490,8.942595,8.0,8.0,8.0,22.716291,0.001986,179.742550,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2008.0,40.698520,19.950300
AL200800000011,0.106758,8.940361,8.0,8.0,8.0,41.684960,0.001609,179.742550,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2008.0,40.717968,19.935875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZW201500000394,0.001635,0.000000,716.0,716.0,716.0,46.712948,0.000329,-17.159021,52.645260,0.011162,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2015.0,-17.929422,30.997536
ZW201500000395,0.007476,0.000000,716.0,716.0,716.0,0.000000,0.000404,-18.019940,51.753593,0.011510,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2015.0,-17.166506,29.718370
ZW201500000396,0.002424,0.000000,716.0,716.0,716.0,80.522360,0.000402,-17.357134,53.740124,0.011052,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2015.0,-17.915289,31.156115


In [9]:
def mcrmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return np.average(rmse)

In [10]:
model = RandomForestRegressor(n_estimators=4000, max_features=0.5, oob_score=mcrmse, random_state=42, n_jobs=-1, verbose=1)
model.fit(X_new, y_new)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   30.2s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   56.1s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 4000 out of 4000 | elapsed:  5.0min finished


RandomForestRegressor(max_features=0.5, n_estimators=4000, n_jobs=-1,
                      oob_score=<function mcrmse at 0x15f886f80>,
                      random_state=42, verbose=1)

In [11]:
test = pd.read_parquet('../data/test.parquet.gzip').drop(columns=low_imp_features)

In [12]:
y_pred = model.predict(test)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    1.5s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    2.9s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    5.4s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    7.5s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:   10.4s
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed:   13.6s
[Parallel(n_jobs=8)]: Done 4000 out of 4000 | elapsed:   17.1s finished


In [13]:
out = pd.DataFrame(y_pred, columns=pred_cols)
out['DHSID'] = test.index
out = out[['DHSID'] + pred_cols]
out.to_csv('../submission/outlier_removed.csv', index=False)